In [7]:
import pandas as pd
import numpy as np
import pandas_gbq
# Import the Goolge Cloud client Library
from google.cloud import bigquery
%load_ext google.cloud.bigquery



The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [8]:
# Set your default project here
pandas_gbq.context.project = 'bigquery-public-data'
pandas_gbq.context.dialect = 'standard'

In [9]:
# Instantiates a client for Bigquery Service
client = bigquery.Client()

In [10]:
sql = """
SELECT
    plurality,
    COUNT(1) AS count,
    year
FROM
    `bigquery-public-data.samples.natality`
WHERE
    NOT IS_NAN(plurality) AND plurality > 1
GROUP BY
    plurality, year
ORDER BY
    count DESC
"""
df = client.query(sql).to_dataframe()
df.head()

,plurality,count,year
0,2,139209,2007
1,2,138866,2008
2,2,137239,2006
3,2,133285,2005
4,2,132344,2004


In [11]:
%%bigquery
SELECT name, SUM(number) as count
FROM `bigquery-public-data.usa_names.usa_1910_current`
GROUP BY name
ORDER BY count DESC
LIMIT 5


ERROR:
 ('invalid_grant: Token has been expired or revoked.', '{\n  "error": "invalid_grant",\n  "error_description": "Token has been expired or revoked."\n}')


In [33]:
orders = pd.read_csv('orders.csv')
orders.head(10)

,id,first_name,last_name,email,shoe_type,shoe_material,shoe_color,price
0,41874,Kyle,Peck,KylePeck71@gmail.com,ballet flats,faux-leather,black,385.0
1,31349,Elizabeth,Velazquez,EVelazquez1971@gmail.com,boots,fabric,brown,388.0
2,43416,Keith,Saunders,KS4047@gmail.com,sandles,leather,navy,346.0
3,56054,Ryan,Sweeney,RyanSweeney14@outlook.com,sandles,fabric,brown,344.0
4,77402,Donna,Blankenship,DB3807@gmail.com,stilettos,fabric,brown,289.0
5,97148,Albert,Dillon,Albert.Dillon@gmail.com,wedges,fabric,brown,266.0
6,19998,Judith,Hewitt,JudithHewitt98@gmail.com,stilettos,leather,black,395.0
7,83290,Kayla,Hardin,Kayla.Hardin@gmail.com,stilettos,leather,white,241.0
8,77867,Steven,Blankenship,Steven.Blankenship@gmail.com,wedges,leather,navy,266.0
9,54885,Carol,Mclaughlin,CM3415@gmail.com,ballet flats,faux-leather,brown,440.0


### Our finance department wants to know the price of the most expensive pair of shoes purchased. Save your answer to the variable most_expensive.

In [34]:
most_expensive = orders.price.max()
print(most_expensive)

493.0


Our fashion department wants to know how many different colors of shoes we are selling. Save your answer to the variable num_colors.

In [35]:
num_colors = orders.shoe_color.nunique()

Let’s return to our orders data from ShoeFly.com.
In the previous exercise, our finance department wanted to know the most expensive shoe that we sold.
Now, they want to know the most expensive shoe for each shoe_type (i.e., the most expensive boot, the most expensive ballet flat, etc.).
Save your answer to the variable pricey_shoes.

Use .reset_index() to convert the result of order.groupby('shoe_type').price.max() to a pandas dataframe

In [43]:
price_shoes = orders.groupby('shoe_type').price.max().reset_index()
display(price_shoes)
print(type(price_shoes))

,shoe_type,price
0,ballet flats,481.0
1,boots,478.0
2,clogs,493.0
3,sandles,456.0
4,stilettos,487.0
5,wedges,461.0


<class 'pandas.core.frame.DataFrame'>


# Calculating Aggregate Functions III

np.percentile can calculate any percentile over an array of values

high_earners = df.groupby('category').wage
    .apply(lambda x: np.percentile(x, 75))
    .reset_index()

In [44]:
cheap_shoes = orders.groupby('shoe_color').price.apply(lambda x: np.percentile(x,25)).reset_index()


display(cheap_shoes)

,shoe_color,price
0,black,NaN
1,brown,193.5
2,navy,205.5
3,red,250.0
4,white,196.0


## Calculating Aggregate Functions IV

In [46]:
shoe_counts=orders.groupby(['shoe_color','shoe_type']).id.count().reset_index()

display(shoe_counts)

,shoe_color,shoe_type,id
0,black,ballet flats,2
1,black,boots,3
2,black,clogs,4
3,black,sandles,1
4,black,stilettos,5
5,black,wedges,3
6,brown,ballet flats,5
7,brown,boots,5
8,brown,clogs,6
9,brown,sandles,4


## Pivot Tables

n Pandas, the command for pivot is:

df.pivot(columns='ColumnToPivot',
         index='ColumnToBeRows',
         values='ColumnToBeValues')

In [48]:
shoe_counts = orders.groupby(['shoe_type', 'shoe_color']).id.count().reset_index()

shoe_counts_pivot = shoe_counts.pivot(columns="shoe_color",
index="shoe_type",
values="id").reset_index()

display(shoe_counts_pivot)

shoe_color,shoe_type,black,brown,navy,red,white
0,ballet flats,2.0,5.0,NaN,3.0,5.0
1,boots,3.0,5.0,6.0,2.0,3.0
2,clogs,4.0,6.0,1.0,4.0,1.0
3,sandles,1.0,4.0,5.0,3.0,4.0
4,stilettos,5.0,3.0,2.0,2.0,2.0
5,wedges,3.0,4.0,4.0,5.0,2.0


In [51]:
user_visits = pd.read_csv('page_visits.csv')
user_visits.head(10)

,id,first_name,last_name,email,month,utm_source
0,10043,Louis,Koch,LouisKoch43@gmail.com,3-Mar,yahoo
1,10150,Bruce,Webb,BruceWebb44@outlook.com,3-Mar,twitter
2,10155,Nicholas,Hoffman,Nicholas.Hoffman@gmail.com,2-Feb,google
3,10178,William,Key,William.Key@outlook.com,3-Mar,yahoo
4,10208,Karen,Bass,KB4971@gmail.com,2-Feb,google
5,10260,Benjamin,Ochoa,Benjamin.Ochoa@outlook.com,1-Jan,twitter
6,10271,Gerald,Aguilar,Gerald.Aguilar@gmail.com,3-Mar,google
7,10278,Melissa,Lambert,Melissa.Lambert@gmail.com,2-Feb,email
8,10320,Adam,Strickland,Adam.Strickland@gmail.com,3-Mar,email
9,10389,Ethan,Payne,EthanPayne26@outlook.com,2-Feb,facebook


In [53]:
click_source = user_visits.groupby('utm_source').id.count().reset_index()


click_source_by_month = user_visits.groupby(['utm_source','month']).id.count().reset_index()

click_source_by_month_pivot = click_source_by_month.pivot(
  columns="month",
  index='utm_source',
  values = 'id'
).reset_index()

display(click_source_by_month_pivot)

month,utm_source,1-Jan,2-Feb,3-Mar
0,email,43,147,272
1,facebook,404,263,156
2,google,127,196,220
3,twitter,164,154,97
4,yahoo,262,240,255
